# Evaluation & Warning Guides: The AlarmEngine

This notebook provides a guide to the `AlarmEngine`, a key component of the `chs-scada-dispatch` service. Its purpose is to continuously monitor the state of the system, check for predefined alarm conditions, and generate events when those conditions are met. This is a critical function for system safety and operational awareness.

## 1. How It Works: Rules and Events

The `AlarmEngine` operates on a simple but powerful principle:

1.  **Loads Rules**: On startup, it loads a set of human-readable rules from a `rules.yaml` file.
2.  **Fetches Data**: Periodically, it fetches the latest status of all devices from the `TimeSeriesDB`.
3.  **Evaluates Conditions**: For each device, it checks if any of the rules apply. A rule's `condition` is a simple Python expression that is evaluated against the device's data.
4.  **Generates Events**: If a rule's condition is met and that alarm is not already active, the engine generates a new alarm event and stores it in the `EventStore`.

### Example `rules.yaml`

The logic is defined entirely in YAML, so no code changes are needed to add new alarms. A typical rule looks like this:

```yaml
- rule_name: "Reservoir High Level Warning"
  device_id_pattern: "res_*"  # Applies to all devices starting with 'res_'
  condition: "values.level > 150.0"
  severity: "WARNING"
  message: "Reservoir {device_id} level {values.level:.2f}m has exceeded the 150.0m warning threshold!"

- rule_name: "Pump Offline Alert"
  device_id_pattern: "pump_station_*"
  condition: "values.pump_status == 0"
  severity: "CRITICAL"
  message: "The pump at {device_id} has gone offline!"
```

## 2. Code Example: Triggering an Alarm

To demonstrate the `AlarmEngine` in a self-contained way, we first need to create **mock** versions of its dependencies (`TimeSeriesDB` and `EventStore`) that work in memory without a real database.

In [ ]:
import sys
import os
import yaml
import time

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

# We need to import the engine itself, but also create mocks for its dependencies
from chs-scada-dispatch.alarm_engine.engine import AlarmEngine

# --- Mock Database Classes for the Tutorial ---

class MockTimeSeriesDB:
    """A mock TimeSeriesDB that just holds a dictionary of statuses."""
    def __init__(self, statuses):
        self._statuses = statuses
    def get_latest_statuses(self):
        return self._statuses

class MockEventStore:
    """A mock EventStore that just holds a list of events in memory."""
    def __init__(self):
        self.events = []
    def add_event(self, event_data):
        print(f"EVENT STORED: {event_data}")
        self.events.append(event_data)


Now, we can set up the test scenario. We will:
1. Create a mock `rules.yaml` file on the fly.
2. Create mock device data where one device violates a rule.
3. Run the `AlarmEngine`'s check cycle once.
4. Verify that the correct alarm was added to our mock `EventStore`.

In [ ]:
# 1. Create mock rules file
rules_content = """
- rule_name: "High Level Warning"
  device_id_pattern: "tank_*"
  condition: "values.level > 50"
  severity: "WARNING"
  message: "Tank {device_id} level is high: {values.level}m."
"""
# The engine expects the file to be in a specific relative path
rules_dir = 'chs-scada-dispatch/alarm_engine'
os.makedirs(rules_dir, exist_ok=True)
with open(os.path.join(rules_dir, 'rules.yaml'), 'w') as f:
    f.write(rules_content)

# 2. Create mock device data
mock_statuses = {
    "tank_01": {"values": {"level": 25.0, "pressure": 10}},
    "tank_02": {"values": {"level": 55.0, "pressure": 30}} # This one should trigger the alarm
}

# 3. Instantiate services
mock_ts_db = MockTimeSeriesDB(mock_statuses)
mock_event_store = MockEventStore()

# The AlarmEngine will load the rules.yaml we just created
alarm_engine = AlarmEngine(timeseries_db=mock_ts_db, event_store=mock_event_store)

# 4. Run one check cycle manually
print("Running alarm check...")
alarm_engine._check_all_devices(mock_statuses)
print("Check complete.")

# 5. Verify the results
print(f"\nTotal events generated: {len(mock_event_store.events)}")
assert len(mock_event_store.events) == 1, "Expected exactly one alarm!"

generated_event = mock_event_store.events[0]
assert generated_event['source'] == 'tank_02', "Alarm source should be tank_02"
assert generated_event['severity'] == 'WARNING', "Alarm severity should be WARNING"
assert "level is high: 55.0m" in generated_event['message'], "Alarm message is incorrect"

print("\nVerification successful! The correct alarm was generated.")

This example shows how the `AlarmEngine` provides a powerful, configuration-driven way to add monitoring and diagnostics to the CHS platform.